# Fields

We have been working with fields in the previous examples already. Now let's take a deeper look at fields in yt, how they are handled, and how to define your own derived fields. 

In [ ]:
import yt
import numpy as np

In [ ]:
ds = yt.load("snapshot_033/snap_033.0.hdf5")

We are first interested in what kinds of "fields" are intrinsic to the dataset. We can list them using `ds.field_list`:

In [ ]:
ds.field_list

We also mentioned "derived" fields earlier. We can see which derived fields are available using `ds.derived_field_list`:

In [ ]:
ds.derived_field_list

Notice that the fields that are in the dataset are part of the derived field list as well--reflecting yt's philosophy on fields.

You can examine various aspects of all fields using the `ds.fields` syntax:

In [ ]:
ds.fields.gas.entropy

In [ ]:
ds.fields.gas.entropy.get_label()

In [ ]:
ds.fields.gas.entropy.get_units()

In [ ]:
print(ds.fields.gas.entropy.get_source())

## Types of Fields

* Mesh Fields: These are fields defined on the mesh. In mesh-based datasets this is self-explanatory, but in particle and octree-based datasets this is the octree. In the case of particle datasets, mesh fields have been smoothed from the particles onto the mesh using an interpolation scheme (the standard SPH smoothing kernel, usually). The field type is typically `"gas"`, but fields belonging to a particular type of dataset have their own field types. 
* Particle Fields: Mesh codes can have particles (FLASH, Enzo, etc.), and of course other codes (e.g. Gadget and its derivatives) are purely particle-based.
* Index Fields: These are fields for coordinates, cell widths, cell volumes, etc. 
* Deposit Fields: Deposit fields are fields that have been mapped onto the mesh from particles. 

## Creating Your Own Derived Fields

yt has lots of fields, but undoubtedly you have some fields of your own that you may want to analyze and visualize. yt provides a way for you create your own fields. 

Let's define a derived field for free-free emission in X-rays. It has this formula (Equation 5.15b from Rybicki and Lightman 1979):

$\epsilon_{\rm ff}~({\rm erg}~{\rm cm}^{-3}) = 1.4 \times 10^{-27}T^{1/2}n_en_iZ^2\bar{g}_B$

First, we need to set up a Python function which will set up this mathematical relationship. Let's define and import some constants that we'll need:

In [ ]:
# Assuming full ionization
mue = 1.136
mui = 1.22
from yt.units import mp
g_B = 1.2 # Approximate Gaunt factor
Z = 1.111 # Approximate atomic number

Next, we'll set up the Python function. Because the function for free-free emission has some numerical factors, it's easiest in this case to strip the units during the calculation using `.v` and `.d` and add them back at the end using `data.ds.arr`:

In [ ]:
def _free_free(field, data):
    n_e = data["gas", "density"].d/(mue*mp.v)
    n_i = data["gas", "density"].d/(mui*mp.v)
    T = data["gas", "temperature"].d
    return data.ds.arr(1.4e-27*np.sqrt(T)*n_e*n_i*Z*Z, "erg/cm**3")

Finally, we use the function `ds.add_field()` to add the field definition to the fields that the dataset knows about. We have to give it a `(type, name)` specification, supply the function, and the units:

In [ ]:
ds.add_field(("gas", "free_free_emission"), function=_free_free, units="erg/cm**3")

Now that we have created this field, we can use it in exactly the same way as any other:

In [ ]:
sp = ds.sphere("max", (1.0, "Mpc"))

In [ ]:
sp.mean("free_free_emission")

In [ ]:
proj = sp.integrate("free_free_emission", axis="x")

In [ ]:
p = proj.plot()

In [ ]:
p.set_center([12.2132301330566406,9.3899726867675781])
p.zoom(128)